In [1]:
import numpy as np
import mxnet as mx
from collections import namedtuple, Counter
from unidecode import unidecode
from mxnet.io import DataIter

import operator
import re
import warnings

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-28 17:07:40.039624. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [2]:
# Get rid of annoying Python deprecation warnings from built-in JSON encoder
warnings.filterwarnings("ignore", category=DeprecationWarning)   

In [3]:
Args = namedtuple(
    'Args', 
    ('test load_epoch num_layers num_hidden num_embed bidirectional gpus '
     'kv_store num_epochs optimizer mom wd lr batch_size disp_batches '
     'stack_rnn dropout model_prefix'))

args = Args(
    test=          False,
    load_epoch=    0,
    num_layers=    2,
    num_hidden=    200,
    num_embed=     200,
    bidirectional= False,
    gpus=          '0,1',
    kv_store=      'device',
    num_epochs=    1,
    optimizer=    'adam',
    mom=           0.9,
    wd=           0.00001,
    lr = 0.001,
    batch_size=    32,
    disp_batches=  50,
    stack_rnn=     False,
    dropout=       0.5,
    model_prefix= 'foo'
)

In [4]:
# Do a word count to get the number of words 

start_label = 1
invalid_label = 0

# Decode text as UTF-8
# Remove diacritical signs and convert to Latin alphabet
# Separate punctuation as separate "words"
def tokenize_text(fname, vocab=None, invalid_label=-1, start_label=0, sep_punctuation=True):
    lines = unidecode(open(fname).read().decode('utf-8')).split('\n')
    lines = [x for x in lines if x]
    lines = map(lambda x: re.findall(r"\w+|[^\w\s]", x, re.UNICODE), lines)    
    sentences, vocab = mx.rnn.encode_sentences(lines, vocab=vocab, invalid_label=invalid_label, start_label=start_label)
    return sentences, vocab

def get_data(src_path, targ_path, start_label=1, invalid_label=0):
    src_sent, src_vocab = tokenize_text(src_path, start_label=start_label,
                                invalid_label=invalid_label)
    
    targ_sent, targ_vocab = tokenize_text(targ_path, start_label=start_label, #new_start+1,
                                          invalid_label=invalid_label)

    return src_sent, src_vocab, targ_sent, targ_vocab

In [13]:
src_sent, src_vocab, targ_sent, targ_vocab = \
    get_data(
        src_path='./data/europarl-v7.pl-en.en', # pl-en.small.en
        targ_path='./data/europarl-v7.pl-en.pl', # pl-en.small.pl
        start_label=1,
        invalid_label=0
    )

In [19]:
def gen_buckets(src_sent, targ_sent, filter_smaller_counts_than=None):
    length_pairs = map(lambda x: (len(x[0]), len(x[1])), zip(src_sent, targ_sent))
    counts = list(Counter(length_pairs).items())
    c_sorted = sorted(counts, key=operator.itemgetter(0, 1))
    buckets = [i for i in c_sorted if i[1] >= filter_smaller_counts_than]
    return buckets

In [20]:
all_buckets = gen_buckets(src_sent, targ_sent, filter_smaller_counts_than=None)

In [21]:
bucket_filter = 32
batched_buckets = gen_buckets(src_sent, targ_sent, filter_smaller_counts_than=bucket_filter)

In [22]:
num_sent = lambda sent: reduce(lambda a, b: a + b, map(lambda x: x[1], sent))
num_all = num_sent(all_buckets)
num_batched = num_sent(batched_buckets)
print("# of all buckets: %d (# sent: %d)" % (len(all_buckets), num_all))
print("# of buckets with counts < %d filtered out: %d (num sent: %d)" % (bucket_filter, len(batched_buckets), num_batched))

# of all buckets: 9024 (# sent: 629529)
# of buckets with counts < 32 filtered out: 2992 (num sent: 591527)


In [23]:
def print_sorted_buckets(buckets):
    b = sorted(buckets, key=operator.itemgetter(0, 1))
    for i in b:
        print(i)

print_sorted_buckets(batched_buckets)

((2, 2), 422)
((2, 3), 37)
((2, 5), 45)
((2, 6), 55)
((2, 7), 55)
((2, 8), 75)
((2, 9), 92)
((2, 10), 99)
((2, 11), 91)
((2, 12), 95)
((2, 13), 105)
((2, 14), 100)
((2, 15), 118)
((2, 16), 120)
((2, 17), 105)
((2, 18), 124)
((2, 19), 101)
((2, 20), 125)
((2, 21), 101)
((2, 22), 91)
((2, 23), 93)
((2, 24), 96)
((2, 25), 93)
((2, 26), 77)
((2, 27), 93)
((2, 28), 86)
((2, 29), 63)
((2, 30), 80)
((2, 31), 69)
((2, 32), 56)
((2, 33), 64)
((2, 34), 46)
((2, 35), 67)
((2, 36), 57)
((2, 37), 48)
((2, 39), 41)
((2, 40), 39)
((3, 2), 66)
((3, 3), 59)
((3, 4), 42)
((3, 5), 42)
((3, 6), 67)
((3, 7), 56)
((3, 8), 84)
((3, 9), 95)
((3, 10), 84)
((3, 11), 90)
((3, 12), 106)
((3, 13), 108)
((3, 14), 97)
((3, 15), 100)
((3, 16), 113)
((3, 17), 107)
((3, 18), 102)
((3, 19), 101)
((3, 20), 112)
((3, 21), 87)
((3, 22), 78)
((3, 23), 93)
((3, 24), 97)
((3, 25), 82)
((3, 26), 82)
((3, 27), 72)
((3, 28), 62)
((3, 29), 60)
((3, 30), 60)
((3, 31), 72)
((3, 32), 55)
((3, 33), 73)
((3, 34), 47)
((3, 35), 43)
((3

In [24]:
class TwoDBisect:
    def __init__(self, buckets):
        self.buckets = sorted(buckets, key=operator.itemgetter(0, 1))
        self.x, self.y = zip(*buckets)
        self.x, self.y = np.array(list(self.x)), np.array(list(self.y))
        
    def twod_bisect(self, source, target):    
        offset1 = np.searchsorted(self.x, len(source), side='left')
        offset2 = np.where(self.y[offset1:] >= len(target))[0]        
        return self.buckets[offset2[0]]   

In [25]:
sent_len = lambda x: map(lambda y: len(y), x)
max_len = lambda x: max(sent_len(x))
min_len = lambda x: min(sent_len(x))

min_len = min(min(sent_len(src_sent)), min(sent_len(targ_sent)))
max_len = max(max(sent_len(src_sent)), max(sent_len(targ_sent)))

# min_len = min
# max_len = 65
increment = 5

all_pairs = [(i, j) for i in range(
        min_len,max_len+increment,increment
    ) for j in range(
        min_len,max_len+increment,increment
    )]


bisect = TwoDBisect(buckets=all_pairs)

In [26]:
bisect

<__main__.TwoDBisect instance at 0x7fcbd22696c8>

In [27]:
all_pairs

[(1, 1),
 (1, 6),
 (1, 11),
 (1, 16),
 (1, 21),
 (1, 26),
 (1, 31),
 (1, 36),
 (1, 41),
 (1, 46),
 (1, 51),
 (1, 56),
 (1, 61),
 (1, 66),
 (1, 71),
 (1, 76),
 (1, 81),
 (1, 86),
 (1, 91),
 (1, 96),
 (1, 101),
 (1, 106),
 (1, 111),
 (1, 116),
 (1, 121),
 (1, 126),
 (1, 131),
 (1, 136),
 (1, 141),
 (1, 146),
 (1, 151),
 (1, 156),
 (1, 161),
 (1, 166),
 (1, 171),
 (1, 176),
 (1, 181),
 (1, 186),
 (1, 191),
 (1, 196),
 (1, 201),
 (1, 206),
 (1, 211),
 (1, 216),
 (1, 221),
 (1, 226),
 (1, 231),
 (1, 236),
 (1, 241),
 (1, 246),
 (1, 251),
 (1, 256),
 (1, 261),
 (1, 266),
 (1, 271),
 (1, 276),
 (1, 281),
 (1, 286),
 (1, 291),
 (1, 296),
 (1, 301),
 (1, 306),
 (1, 311),
 (1, 316),
 (1, 321),
 (1, 326),
 (1, 331),
 (1, 336),
 (1, 341),
 (6, 1),
 (6, 6),
 (6, 11),
 (6, 16),
 (6, 21),
 (6, 26),
 (6, 31),
 (6, 36),
 (6, 41),
 (6, 46),
 (6, 51),
 (6, 56),
 (6, 61),
 (6, 66),
 (6, 71),
 (6, 76),
 (6, 81),
 (6, 86),
 (6, 91),
 (6, 96),
 (6, 101),
 (6, 106),
 (6, 111),
 (6, 116),
 (6, 121),
 (6, 126),

In [32]:
tuples = []

for src, targ in zip(src_sent, targ_sent):
    try:
        len_tup = bisect.twod_bisect(src, targ)
        tuples.append(len_tup)
    except Exception as e:
        print("src length: %d" % len(src))
        print("targ length: %d" % len(targ))

In [33]:
tuples

[(1, 11),
 (1, 6),
 (1, 11),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 16),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 11),
 (1, 6),
 (1, 21),
 (1, 6),
 (1, 21),
 (1, 6),
 (1, 21),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 21),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 6),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 16),
 (1, 11),
 (1, 26),
 (1, 6),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 11),
 (1, 6),
 (1, 6),
 (1, 16),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 6),
 (1, 6),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 21),
 (1, 6),
 (1, 21),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 16),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 6),
 (1, 11),
 (1, 6

In [30]:
counts = list(Counter(tuples).items())
batch_size = 32
filter_smaller_counts_than = batch_size

c_sorted = sorted(counts, key=operator.itemgetter(0, 1))
buckets = [i for i in c_sorted if i[1] >= filter_smaller_counts_than]
print(buckets)

[((1, 1), 327), ((1, 6), 36490), ((1, 11), 81209), ((1, 16), 103065), ((1, 21), 104410), ((1, 26), 90820), ((1, 31), 69602), ((1, 36), 50062), ((1, 41), 33493), ((1, 46), 22014), ((1, 51), 14047), ((1, 56), 8728), ((1, 61), 5486), ((1, 66), 3525), ((1, 71), 2162), ((1, 76), 1341), ((1, 81), 896), ((1, 86), 558), ((1, 91), 377), ((1, 96), 271), ((1, 101), 159), ((1, 106), 122), ((1, 111), 82), ((1, 116), 67), ((1, 121), 39), ((1, 126), 38)]


In [31]:
class TwoDBisect:
    def __init__(self, buckets):
        self.buckets = sorted(buckets, key=operator.itemgetter(0, 1))
        self.x, self.y = zip(*buckets)
        self.x, self.y = np.array(list(self.x)), np.array(list(self.y))
        
    def twod_bisect(self, source, target):    
        offset1 = np.searchsorted(self.x, len(source), side='left')
        offset2 = np.where(self.y[offset1:] >= len(target))[0]        
        return self.buckets[offset2[0]]   

In [ ]:
class BucketSeq2SeqIter(DataIter):
    """Simple bucketing iterator for sequence-to-sequence models.
    
    @staticmethod
    def 
    
    Parameters
    ----------
    source_seq : list of list of int
    target_seq : list of list of int
    batch_size : int
        batch_size of data    
    invalid_label : int, default -1
        key for invalid label, e.g. <end-of-sentence>
    dtype : str, default 'float32'
        data type
    buckets : list of pairs of int (source seq length, target seq length)
        size of data buckets. Automatically generated if None.
    data_name : str, default 'data'
        name of data    print(buckets)
    label_name : str, default 'softmax_label'
        name of label
    layout : str
        format of data and label. 'NT' means (batch_size, length)
        and 'TN' means (length, batch_size).
    """
    def __init__(self, source, target, batch_size, buckets=None, invalid_label=-1,
                 data_name='data', label_name='softmax_label', dtype='float32',
                 layout='NTC', min_sent_len = 1, max_sent_len = 60):
        super(BucketSeq2SeqIter, self).__init__()
        
        # insert call to gen_buckets here if buckets are not provided
        if not buckets:
            all_pairs = [(i, j) for i in range(
                min_len,max_len+increment,increment
            ) for j in range(
                min_len,max_len+increment,increment
            )]
            self.bisect = TwoDBisect(buckets = all_pairs)
            buckets = 
            
#             [i for i, j in enumerate(np.bincount([len(s) for s in sentences]))
#                        if j >= batch_size]
            
        # Sorting is kinda pointless because it's first by the first element of the tuple,
        # then the next. So, it could be [(1, 2), (1, 20), (2, 5), (2, 71)]
#         buckets.sort()

        ndiscard = 0
        self.data = [[] for _ in buckets]
        for i, sent in enumerate(sentences):
            
            # this bisect also won't work because it's now based on a tuple of lengths
            buck = bisect.bisect_left(buckets, len(sent))
            # this test is not appropriate because of the tuple of lengths
            if buck == len(buckets):
                ndiscard += 1
                continue
            buff = np.full((buckets[buck],), invalid_label, dtype=dtype)
            buff[:len(sent)] = sent
            self.data[buck].append(buff)

        self.data = [np.asarray(i, dtype=dtype) for i in self.data]

        print("WARNING: discarded %d sentences longer than the largest bucket."%ndiscard)

        self.batch_size = batch_size
        self.buckets = buckets
        self.data_name = data_name
        self.label_name = label_name
        self.dtype = dtype
        self.invalid_label = invalid_label
        self.nddata = []
        self.ndlabel = []
        self.major_axis = layout.find('N')
        self.default_bucket_key = max(buckets)

        if self.major_axis == 0:
            self.provide_data = [(data_name, (batch_size, self.default_bucket_key))]
            self.provide_label = [(label_name, (batch_size, self.default_bucket_key))]
        elif self.major_axis == 1:
            self.provide_data = [(data_name, (self.default_bucket_key, batch_size))]
            self.provide_label = [(label_name, (self.default_bucket_key, batch_size))]
        else:
            raise ValueError("Invalid layout %s: Must by NT (batch major) or TN (time major)")

        self.idx = []
        for i, buck in enumerate(self.data):
            self.idx.extend([(i, j) for j in range(0, len(buck) - batch_size + 1, batch_size)])
        self.curr_idx = 0

        self.reset()

    def reset(self):
        self.curr_idx = 0
        random.shuffle(self.idx)
        for buck in self.data:
            np.random.shuffle(buck)

        self.nddata = []
        self.ndlabel = []
        for buck in self.data:
            label = np.empty_like(buck)
            label[:, :-1] = buck[:, 1:]
            label[:, -1] = self.invalid_label
            self.nddata.append(ndarray.array(buck, dtype=self.dtype))
            self.ndlabel.append(ndarray.array(label, dtype=self.dtype))

    def next(self):
        if self.curr_idx == len(self.idx):
            raise StopIteration
        i, j = self.idx[self.curr_idx]
        self.curr_idx += 1

        if self.major_axis == 1:
            data = self.nddata[i][j:j+self.batch_size].T
            label = self.ndlabel[i][j:j+self.batch_size].T
        else:
            data = self.nddata[i][j:j+self.batch_size]
            label = self.ndlabel[i][j:j+self.batch_size]

        return DataBatch([data], [label],
                         bucket_key=self.buckets[i],
                         provide_data=[(self.data_name, data.shape)],
                         provide_label=[(self.label_name, label.shape)])

In [ ]:
print(batched_buckets)

In [ ]:
buckets = list(range(5, 60, 5))
#[10, 20, 30, 40, 50, 60]

                 data_name='data', label_name='softmax_label', dtype='float32',
start_label = 1
invalid_label = 0

def tokenize_text(fname, vocab=None, invalid_label=-1, start_label=0):
    lines = open(fname).readlines()
    lines = [filter(None, i.split(' ')) for i in lines]
    sentences, vocab = mx.rnn.encode_sentences(lines, vocab=vocab, invalid_label=invalid_label, start_label=start_label)
    return sentences, vocab

def get_data(layout):
    train_sent, vocab = tokenize_text("./data/ptb.train.txt", start_label=start_label,
                                      invalid_label=invalid_label)
    val_sent, _ = tokenize_text("./data/ptb.test.txt", vocab=vocab, start_label=start_label,
                                invalid_label=invalid_label)

    data_train  = mx.rnn.BucketSentenceIter(train_sent, args.batch_size, buckets=buckets,
                                            invalid_label=invalid_label, layout=layout)
    data_val    = mx.rnn.BucketSentenceIter(val_sent, args.batch_size, buckets=buckets,
                                            invalid_label=invalid_label, layout=layout)
    
    print("default: %s" % data_train.default_bucket_key)
    return data_train, data_val, vocab


def train(args):
    data_train, data_val, vocab = get_data('TN')
    if args.stack_rnn:
        stack = mx.rnn.SequentialRNNCell()
        for layer in range(args.num_layers):
            dropout = 0.0
            if layer < (args.num_layers - 1):
                dropout = args.dropout
            stack.add(mx.rnn.FusedRNNCell(args.num_hidden, num_layers=1,
                    mode='lstm', prefix='lstm_%d'%layer, dropout=dropout,
                    bidirectional=args.bidirectional))
        cell = stack
    else:
        cell = mx.rnn.FusedRNNCell(args.num_hidden, num_layers=args.num_layers, dropout=args.dropout,
                mode='lstm', bidirectional=args.bidirectional)

    def sym_gen(seq_len):
        data = mx.sym.Variable('data')
        label = mx.sym.Variable('softmax_label')
        embed = mx.sym.Embedding(data=data, input_dim=len(vocab), output_dim=args.num_embed,name='embed')

        output, _ = cell.unroll(seq_len, inputs=embed, merge_outputs=True, layout='TNC')

        pred = mx.sym.Reshape(output,
                shape=(-1, args.num_hidden*(1+args.bidirectional)))
        pred = mx.sym.FullyConnected(data=pred, num_hidden=len(vocab), name='pred')

        label = mx.sym.Reshape(label, shape=(-1,))
        pred = mx.sym.SoftmaxOutput(data=pred, label=label, name='softmax')

        return pred, ('data',), ('softmax_label',)

    if args.gpus:
        contexts = [mx.gpu(int(i)) for i in args.gpus.split(',')]
    else:
        contexts = mx.cpu(0)

    model = mx.mod.BucketingModule(
        sym_gen             = sym_gen,
        default_bucket_key  = data_train.default_bucket_key,
        context             = contexts)

    if args.load_epoch:
        _, arg_params, aux_params = mx.rnn.load_rnn_checkpoint(
            cell, args.model_prefix, args.load_epoch)
    else:
        arg_params = None
        aux_params = None

    opt_params = {
      'learning_rate': args.lr,
      'wd': args.wd
    }

    if args.optimizer not in ['adadelta', 'adagrad', 'adam', 'rmsprop']:
        opt_params['momentum'] = args.mom

    model.fit(
        train_data          = data_train,
        eval_data           = data_val,
        eval_metric         = mx.metric.Perplexity(invalid_label),
        kvstore             = args.kv_store,
        optimizer           = args.optimizer,
        optimizer_params    = opt_params, 
        initializer         = mx.init.Xavier(factor_type="in", magnitude=2.34),
        arg_params          = arg_params,
        aux_params          = aux_params,
        begin_epoch         = args.load_epoch,
        num_epoch           = args.num_epochs,
        batch_end_callback  = mx.callback.Speedometer(args.batch_size, args.disp_batches),
        epoch_end_callback  = mx.rnn.do_rnn_checkpoint(cell, args.model_prefix, 1)
                              if args.model_prefix else None)

def test(args):
    assert args.model_prefix, "Must specifiy path to load from"
    _, data_val, vocab = get_data('NT')

    if not args.stack_rnn:
        stack = mx.rnn.FusedRNNCell(args.num_hidden, num_layers=args.num_layers,
                mode='lstm', bidirectional=args.bidirectionclass BucketSentenceIter(DataIter):
    """Simple bucketing iterator for language model.
    Label for each step is constructed from data of
    next step.

    Parameters
    ----------
    sentences : list of list of int
        encoded sentences
    batch_size : int
        batch_size of data
    invalid_label : int, default -1
        key for invalid label, e.g. <end-of-sentence>
    dtype : str, default 'float32'
        data type
    buckets : list of int
        size of data buckets. Automatically generated if None.
    data_name : str, default 'data'
        name of data
    label_name : str, default 'softmax_label'
        name of label
    layout : str
        format of data and label. 'NT' means (batch_size, length)
        and 'TN' means (length, batch_size).
    """
    def __init__(self, sentences, batch_size, buckets=None, invalid_label=-1,
                 data_name='data', label_name='softmax_label', dtype='float32',
                 layout='NTC'):
        super(BucketSentenceIter, self).__init__()
        if not buckets:
            buckets = [i for i, j in enumerate(np.bincount([len(s) for s in sentences]))
                       if j >= batch_size]
        buckets.sort()

        ndiscard = 0
        self.data = [[] for _ in buckets]
        for i, sent in enumerate(sentences):
            buck = bisect.bisect_left(buckets, len(sent))
            if buck == len(buckets):
                ndiscard += 1
                continue
            buff = np.full((buckets[buck],), invalid_label, dtype=dtype)
            buff[:len(sent)] = sent
            self.data[buck].append(buff)

        self.data = [np.asarray(i, dtype=dtype) for i in self.data]

        print("WARNING: discarded %d sentences longer than the largest bucket."%ndiscard)

        self.batch_size = batch_size
        self.buckets = buckets
        self.data_name = data_name
        self.label_name = label_name
        self.dtype = dtype
        self.invalid_label = invalid_label
        self.nddata = []
        self.ndlabel = []
        self.major_axis = layout.find('N')
        self.default_bucket_key = max(buckets)

        if self.major_axis == 0:
            self.provide_data = [(data_name, (batch_size, self.default_bucket_key))]
            self.provide_label = [(label_name, (batch_size, self.default_bucket_key))]
        elif self.major_axis == 1:
            self.provide_data = [(data_name, (self.default_bucket_key, batch_size))]
            self.provide_label = [(label_name, (self.default_bucket_key, batch_size))]
        else:
            raise ValueError("Invalid layout %s: Must by NT (batch major) or TN (time major)")

        self.idx = []
        for i, buck in enumerate(self.data):
            self.idx.extend([(i, j) for j in range(0, len(buck) - batch_size + 1, batch_size)])
        self.curr_idx = 0

        self.reset()

    def reset(self):
        self.curr_idx = 0
        random.shuffle(self.idx)
        for buck in self.data:
            np.random.shuffle(buck)

        self.nddata = []
        self.ndlabel = []
        for buck in self.data:
            label = np.empty_like(buck)
            label[:, :-1] = buck[:, 1:]
            label[:, -1] = self.invalid_label
            self.nddata.append(ndarray.array(buck, dtype=self.dtype))
            self.ndlabel.append(ndarray.array(label, dtype=self.dtype))

    def next(self):
        if self.curr_idx == len(self.idx):
            raise StopIteration
        i, j = self.idx[self.curr_idx]
        self.curr_idx += 1

        if self.major_axis == 1:
            data = self.nddata[i][j:j+self.batch_size].T
            label = self.ndlabel[i][j:j+self.batch_size].T
        else:
            data = self.nddata[i][j:j+self.batch_size]
            label = self.ndlabel[i][j:j+self.batch_size]

        return DataBatch([data], [label],
                         bucket_key=self.buckets[i],
                         provide_data=[(self.data_name, data.shape)],
                         provide_label=[(self.label_name, label.shape)])al).unfuse()
    else:
        stack = mx.rnn.SequentialRNNCell()
        for i in range(args.num_layers):
            cell = mx.rnn.LSTMCell(num_hidden=args.num_hidden, prefix='lstm_%dl0_'%i)
            if args.bidirectional:
                cell = mx.rnn.BidirectionalCell(
                        cell,
                        mx.rnn.LSTMCell(num_hidden=args.num_hidden, prefix='lstm_%dr0_'%i),
                        output_prefix='bi_lstm_%d'%i)
            stack.add(cell)

    def sym_gen(seq_len):
        data = mx.sym.Variable('data')
        label = mx.sym.Variable('softmax_label')
        embed = mx.sym.Embedding(data=data, input_dim=len(vocab),
                                 output_dim=args.num_embed, name='embed')

        stack.reset()
        outputs, states = stack.unroll(seq_len, inputs=embed, merge_outputs=True)

        pred = mx.sym.Reshape(outputs,
                shape=(-1, args.num_hidden*(1+args.bidirectional)))
        pred = mx.sym.FullyConnected(data=pred, num_hidden=len(vocab), name='pred')

        label = mx.sym.Reshape(label, shape=(-1,))
        pred = mx.sym.SoftmaxOutput(data=pred, label=label, name='softmax')

        return pred, ('data',), ('softmax_label',)
    
class BucketSentenceIter(DataIter):
    """Simple bucketing iterator for language model.
    Label for each step is constructed from data of
    next step.

    Parameters
    ----------
    sentences : list of list of int
        encoded sentences
    batch_size : int
        batch_size of data
    invalid_label : int, default -1
        key for invalid label, e.g. <end-of-sentence>
    dtype : str, default 'float32'
        data type
    buckets : list of int
        size of data buckets. Automatically generated if None.
    data_name : str, default 'data'
        name of data
    label_name : str, default 'softmax_label'
        name of label
    layout : str
        format of data and label. 'NT' means (batch_size, length)
        and 'TN' means (length, batch_size).
    """
    def __init__(self, sentences, batch_size, buckets=None, invalid_label=-1,
                 data_name='data', label_name='softmax_label', dtype='float32',
                 layout='NTC'):
        super(BucketSentenceIter, self).__init__()
        if not buckets:
            buckets = [i for i, j in enumerate(np.bincount([len(s) for s in sentences]))
                       if j >= batch_size]
        buckets.sort()

        ndiscard = 0
        self.data = [[] for _ in buckets]
        for i, sent in enumerate(sentences):
            buck = bisect.bisect_left(buckets, len(sent))
            if buck == len(buckets):
                ndiscard += 1
                continue
            buff = np.full((buckets[buck],), invalid_label, dtype=dtype)
            buff[:len(sent)] = sent
            self.data[buck].append(buff)

        self.data = [np.asarray(i, dtype=dtype) for i in self.data]

        print("WARNING: discarded %d sentences longer than the largest bucket."%ndiscard)

        self.batch_size = batch_size
        self.buckets = buckets
        self.data_name = data_name
        self.label_name = label_name
        self.dtype = dtype
        self.invalid_label = invalid_label
        self.nddata = []
        self.ndlabel = []
        self.major_axis = layout.find('N')
        self.default_bucket_key = max(buckets)

        if self.major_axis == 0:
            self.provide_data = [(data_name, (batch_size, self.default_bucket_key))]
            self.provide_label = [(label_name, (batch_size, self.default_bucket_key))]
        elif self.major_axis == 1:
            self.provide_data = [(data_name, (self.default_bucket_key, batch_size))]
            self.provide_label = [(label_name, (self.default_bucket_key, batch_size))]
        else:
            raise ValueError("Invalid layout %s: Must by NT (batch major) or TN (time major)")

        self.idx = []
        for i, buck in enumerate(self.data):
            self.idx.extend([(i, j) for j in range(0, len(buck) - batch_size + 1, batch_size)])
        self.curr_idx = 0

        self.reset()

    def reset(self):
        self.curr_idx = 0
        random.shuffle(self.idx)
        for buck in self.data:
            np.random.shuffle(buck)

        self.nddata = []
        self.ndlabel = []
        for buck in self.data:
            label = np.empty_like(buck)
            label[:, :-1] = buck[:, 1:]
            label[:, -1] = self.invalid_label
            self.nddata.append(ndarray.array(buck, dtype=self.dtype))
            self.ndlabel.append(ndarray.array(label, dtype=self.dtype))

    def next(self):
        if self.curr_idx == len(self.idx):
            raise StopIteration
        i, j = self.idx[self.curr_idx]
        self.curr_idx += 1

        if self.major_axis == 1:
            data = self.nddata[i][j:j+self.batch_size].T
            label = self.ndlabel[i][j:j+self.batch_size].T
        else:
            data = self.nddata[i][j:j+self.batch_size]
            label = self.ndlabel[i][j:j+self.batch_size]

        return DataBatch([data], [label],
                         bucket_key=self.buckets[i],
                         provide_data=[(self.data_name, data.shape)],
                         provide_label=[(self.label_name, label.shape)])

    if args.gpus:
        contexts = [mx.gpu(int(i)) for i in args.gpus.split(',')]
    else:
        contexts = mx.cpu(0)

    model = mx.mod.BucketingModule(
        sym_gen             = sym_gen,
        default_bucket_key  = data_val.default_bucket_key,
        context             = contexts)
    model.bind(data_val.provide_data, data_val.provide_label, for_training=False)

    # note here we load using SequentialRNNCell instead of FusedRNNCell.
    _, arg_params, aux_params = mx.rnn.load_rnn_checkpoint(stack, args.model_prefix, args.load_epoch)
    model.set_params(arg_params, aux_params)

    model.score(data_val, mx.metric.Perplexity(invalid_label),
                batch_end_callback=mx.callback.Speedometer(args.batch_size, 5))

In [ ]:
import logging
head = '%(asctime)-15s %(message)s'
logging.basicConfig(level=logging.DEBUG, format=head)

# args = parser.parse_args()

if args.num_layers >= 4 and len(args.gpus.split(',')) >= 4 and not args.stack_rnn:
    print('WARNING: stack-rnn is recommended to train complex model on multiple GPUs')

if args.test:
    # Demonstrates how to load a model trained with CuDNN RNN and predict
    # with non-fused MXNet symbol
    test(args)
else:
    train(args)

In [16]:
class BucketSeq2SeqIter(DataIter):
    """Simple bucketing iterator for sequence-to-sequence models.

    Parameters
    ----------
    sentences : list of list of pairs of int
        encoded sentences (source seq dict int, target seq dict int)
    batch_size : int
        batch_size of data    
    invalid_label : int, default -1
        key for invalid label, e.g. <end-of-sentence>
    dtype : str, default 'float32'
        data type
    buckets : list of pairs of int (source seq length, target seq length)
        size of data buckets. Automatically generated if None.
    data_name : str, default 'data'
        name of data    print(buckets)
    label_name : str, default 'softmax_label'
        name of label
    layout : str
        format of data and label. 'NT' means (batch_size, length)
        and 'TN' means (length, batch_size).
    """
    def __init__(self, input_seq, output_seq, batch_size, buckets=None, invalid_label=-1,
                 data_name='data', label_name='softmax_label', dtype='float32',
                 layout='NTC'):
        super(BucketSentenceIter, self).__init__()
        if not buckets:
            buckets = [i for i, j in enumerate(np.bincount([len(s) for s in sentences]))
                       if j >= batch_size]
        buckets.sort()

        ndiscard = 0
        self.data = [[] for _ in buckets]
        for i, sent in enumerate(sentences):
            buck = bisect.bisect_left(buckets, len(sent))
            if buck == len(buckets):
                ndiscard += 1
                continue
            buff = np.full((buckets[buck],), invalid_label, dtype=dtype)
            buff[:len(sent)] = sent
            self.data[buck].append(buff)

        self.data = [np.asarray(i, dtype=dtype) for i in self.data]

        print("WARNING: discarded %d sentences longer than the largest bucket."%ndiscard)

        self.batch_size = batch_size
        self.buckets = buckets
        self.data_name = data_name
        self.label_name = label_name
        self.dtype = dtype
        self.invalid_label = invalid_label
        self.nddata = []
        self.ndlabel = []
        self.major_axis = layout.find('N')
        self.default_bucket_key = max(buckets)

        if self.major_axis == 0:
            self.provide_data = [(data_name, (batch_size, self.default_bucket_key))]
            self.provide_label = [(label_name, (batch_size, self.default_bucket_key))]
        elif self.major_axis == 1:
            self.provide_data = [(data_name, (self.default_bucket_key, batch_size))]
            self.provide_label = [(label_name, (self.default_bucket_key, batch_size))]
        else:
            raise ValueError("Invalid layout %s: Must by NT (batch major) or TN (time major)")

        self.idx = []
        for i, buck in enumerate(self.data):
            self.idx.extend([(i, j) for j in range(0, len(buck) - batch_size + 1, batch_size)])
        self.curr_idx = 0

        self.reset()

    def reset(self):
        self.curr_idx = 0
        random.shuffle(self.idx)
        for buck in self.data:
            np.random.shuffle(buck)

        self.nddata = []
        self.ndlabel = []
        for buck in self.data:
            label = np.empty_like(buck)
            label[:, :-1] = buck[:, 1:]
            label[:, -1] = self.invalid_label
            self.nddata.append(ndarray.array(buck, dtype=self.dtype))
            self.ndlabel.append(ndarray.array(label, dtype=self.dtype))

    def next(self):
        if self.curr_idx == len(self.idx):
            raise StopIteration
        i, j = self.idx[self.curr_idx]
        self.curr_idx += 1

        if self.major_axis == 1:
            data = self.nddata[i][j:j+self.batch_size].T
            label = self.ndlabel[i][j:j+self.batch_size].T
        else:
            data = self.nddata[i][j:j+self.batch_size]
            label = self.ndlabel[i][j:j+self.batch_size]

        return DataBatch([data], [label],
                         bucket_key=self.buckets[i],
                         provide_data=[(self.data_name, data.shape)],
                         provide_label=[(self.label_name, label.shape)])

In [17]:
class AttentionEncoderCell(BaseRNNCell):
   """Place holder cell that prepare input for attention decoders"""
   def __init__(self, prefix='encode_', params=None):
       super(AttentionEncoderCell, self).__init__(prefix, params=params)

   @property
   def state_shape(self):
       return []

   def __call__(self, inputs, states):
       return inputs, states + [symbol.expand_dims(inputs, axis=1)]

   def unroll(self, length, inputs, begin_state=None, layout='NTC', merge_outputs=None):
       outputs = _normalize_sequence(length, inputs, layout, merge_outputs)
       if merge_outputs is True:
           states = outputs
       else:
           states = inputs

       # attention cell always use NTC layout for states
       states, _ = _normalize_sequence(None, states, 'NTC', True, layout)
       return outputs, [states]


def _attention_pooling(source, scores):
   # source: (batch_size, seq_len, encoder_num_hidden)
   # scores: (batch_size, seq_len, 1)
   probs = symbol.softmax(scores, axis=1)
   output = symbol.batch_dot(source, probs, transpose_a=True)
   return symbol.reshape(output, shape=(0, 0))


class BaseAttentionCell(BaseRNNCell):
   """Base class for attention cells"""
   def __init__(self, prefix='att_', params=None):
       super(BaseAttentionCell, self).__init__(prefix, params=params)

   @property
   def state_shape(self):
       return [(0, 0, 0)]

   def __call__(self, inputs, states):
       raise NotImplementedError


class DotAttentionCell(BaseAttentionCell):
   """Dot attention"""
   def __init__(self, prefix='dotatt_', params=None):
       super(DotAttentionCell, self).__init__(prefix, params=params)

   def __call__(self, inputs, states):
       # inputs: (batch_size, decoder_num_hidden)
       # for dot attention decoder_num_hidden must equal encoder_num_hidden
       if len(states) > 1:
           states = [symbol.concat(*states, dim=1)]

       # source: (batch_size, seq_len, encoder_num_hidden)
       source = states[0]
       # (batch_size, decoder_num_hidden, 1)
       inputs = symbol.expand_dims(inputs, axis=2)
       # (batch_size, seq_len, 1)
       scores = symbol.batch_dot(source, inputs)
       # (batch_size, encoder_num_hidden)
       return _attention_pooling(source, scores), states

NameError: name 'BaseRNNCell' is not defined

In [ ]:
class AttentionEncoderCell(BaseRNNCell):
   """Place holder cell that prepare input for attention decoders"""
   def __init__(self, prefix='encode_', params=None):
       super(AttentionEncoderCell, self).__init__(prefix, params=params)

   @property
   def state_shape(self):
       return []

   def __call__(self, inputs, states):
       return inputs, states + [symbol.expand_dims(inputs, axis=1)]

   def unroll(self, length, inputs, begin_state=None, layout='NTC', merge_outputs=None):
       outputs = _normalize_sequence(length, inputs, layout, merge_outputs)
       if merge_outputs is True:
           states = outputs
       else:
           states = inputs

       # attention cell always use NTC layout for states
       states, _ = _normalize_sequence(None, states, 'NTC', True, layout)
       return outputs, [states]


def _attention_pooling(source, scores):
   # source: (batch_size, seq_len, encoder_num_hidden)
   # scores: (batch_size, seq_len, 1)
   probs = symbol.softmax(scores, axis=1)
   output = symbol.batch_dot(source, probs, transpose_a=True)
   return symbol.reshape(output, shape=(0, 0))


class BaseAttentionCell(BaseRNNCell):
   """Base class for attention cells"""
   def __init__(self, prefix='att_', params=None):
       super(BaseAttentionCell, self).__init__(prefix, params=params)

   @property
   def state_shape(self):
       return [(0, 0, 0)]

   def __call__(self, inputs, states):
       raise NotImplementedError


class DotAttentionCell(BaseAttentionCell):
   """Dot attention"""
   def __init__(self, prefix='dotatt_', params=None):
       super(DotAttentionCell, self).__init__(prefix, params=params)

   def __call__(self, inputs, states):
       # inputs: (batch_size, decoder_num_hidden)
       # for dot attention decoder_num_hidden must equal encoder_num_hidden
       if len(states) > 1:
           states = [symbol.concat(*states, dim=1)]

       # source: (batch_size, seq_len, encoder_num_hidden)
       source = states[0]
       # (batch_size, decoder_num_hidden, 1)
       inputs = symbol.expand_dims(inputs, axis=2)
       # (batch_size, seq_len, 1)
       scores = symbol.batch_dot(source, inputs)
       # (batch_size, encoder_num_hidden)
       return _attention_pooling(source, scores), states